In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
from IPython.display import display, Markdown, Latex
import numpy as np

import glob
frac = 0.01
# path ='./data/RFReg_' # use your path
path ='./../data/Pobreza/pobreza_' # use your path

allFiles = []
allFiles += glob.glob(path+str(frac)+'_q*.csv')
allFiles = sorted(allFiles)
allFiles

['./../data/Pobreza/pobreza_0.01_q2010-02-15.csv',
 './../data/Pobreza/pobreza_0.01_q2010-05-15.csv',
 './../data/Pobreza/pobreza_0.01_q2010-08-15.csv',
 './../data/Pobreza/pobreza_0.01_q2010-11-15.csv',
 './../data/Pobreza/pobreza_0.01_q2011-02-15.csv',
 './../data/Pobreza/pobreza_0.01_q2011-05-15.csv',
 './../data/Pobreza/pobreza_0.01_q2011-08-15.csv',
 './../data/Pobreza/pobreza_0.01_q2011-11-15.csv',
 './../data/Pobreza/pobreza_0.01_q2012-02-15.csv',
 './../data/Pobreza/pobreza_0.01_q2012-05-15.csv',
 './../data/Pobreza/pobreza_0.01_q2012-08-15.csv',
 './../data/Pobreza/pobreza_0.01_q2012-11-15.csv',
 './../data/Pobreza/pobreza_0.01_q2013-02-15.csv',
 './../data/Pobreza/pobreza_0.01_q2013-05-15.csv',
 './../data/Pobreza/pobreza_0.01_q2013-08-15.csv',
 './../data/Pobreza/pobreza_0.01_q2013-11-15.csv',
 './../data/Pobreza/pobreza_0.01_q2014-02-15.csv',
 './../data/Pobreza/pobreza_0.01_q2014-05-15.csv',
 './../data/Pobreza/pobreza_0.01_q2014-08-15.csv',
 './../data/Pobreza/pobreza_0.0

In [3]:
cpi_M = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/IPC-Argentina/main/data/info/indice_precios_M.csv', index_col=0)
cpi_M.index = pd.to_datetime(cpi_M.index)

cpi_Q = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/IPC-Argentina/main/data/info/indice_precios_Q.csv', index_col=0)
cpi_Q.index = pd.to_datetime(cpi_Q.index)

cpi_actual = cpi_M.loc['2022-05']['index'][0]
cpi_2010Q4 = cpi_Q.loc['2010-11']['index'][0]
cpi_ref = cpi_M.loc['2016-01']['index'][0]

In [4]:
aglo_labels = pd.read_csv('./../data/info/aglo_labels.csv')

In [5]:
# df_list = []
# for f in allFiles:
#     print(f)
    
f = './../data/Pobreza/pobreza_0.01_q2010-11-15.csv'
groupers = ['AGLOMERADO', 'PROV', 'NOMPROV']
vars_ = ['INGRESO', 'P47T_persona']

data = pd.read_csv(f, usecols = ['Q'] + groupers + vars_)
data = data.merge(aglo_labels)
data['AGLO_si'] = data.AGLOMERADO != 0
data['Total'] = True

data = data.loc[data.AGLO_si == 1]

data.loc[data.INGRESO == 0, 'decil_P47T'] = 'sin ingresos'
data.loc[data.INGRESO == 1, 'P47T_persona'] = data.loc[data.INGRESO == 1, 'P47T_persona'] + np.random.uniform(0, .01, data.INGRESO.sum())
data.loc[data.INGRESO == 1, 'decil_P47T'] = pd.qcut(data.loc[data.INGRESO == 1, 'P47T_persona'], q = 10, labels = range(1, 11))

data['Q'] = pd.to_datetime(data['Q'])
# df104 = df.loc[df.Q == '2010-11-15']

In [6]:
# df104 = df.loc[df.Q == '2010-11-15']

In [7]:
# pd.read_csv(f, nrows = 5).T

In [8]:
## Tabla:

# Personas con ingresos (aprox 14.377, sin ingresos 10.510 para un total de 24.887 en aglos)


In [9]:
tabla = data.groupby(['decil_P47T']).agg({'P47T_persona': ['min', 'max', 'size', 'sum', 'mean']})['P47T_persona']
tabla[['min', 'max', 'sum', 'mean']] = tabla[['min', 'max', 'sum', 'mean']]*(cpi_2010Q4/cpi_ref)
tabla[['size', 'sum']] = 100*tabla[['size', 'sum']]
tabla['pct'] = 100*tabla['sum']/tabla['sum'].sum()
tabla.loc['Total'] = tabla.iloc[:-1].agg({'min': 'min', 'max': 'max','size': 'sum', 'sum': 'sum', 'mean': 'mean', 'pct': 'sum'})

tabla[['min', 'max', 'sum', 'mean']] = tabla[['min', 'max', 'sum', 'mean']].round(-1).astype(int)
tabla['size'] = (tabla['size']/1e3).round(1)
tabla['sum'] = (tabla['sum']/1e6).round(1)
tabla['pct'] = tabla['pct'].round(1)
tabla.columns = ['Mínimo', 'Máximo', 'Población (miles)', 'Ingresos (mill. ARS)', 'Media', '% de suma de ingresos']
tabla

,Mínimo,Máximo,Población (miles),Ingresos (mill. ARS),Media,% de suma de ingresos
decil_P47T,,,,,,
1,30,500,1555.1,511.8,330,1.4
2,500,800,1555.0,1045.2,670,2.8
3,800,1040,1555.0,1463.5,940,3.9
4,1040,1400,1555.0,1862.3,1200,5.0
5,1400,1750,1555.1,2406.2,1550,6.5
6,1750,2030,1555.0,3008.3,1930,8.1
7,2030,2700,1555.0,3754.9,2410,10.1
8,2700,3500,1555.0,4729.8,3040,12.7
9,3500,5000,1555.0,6422.1,4130,17.2


In [10]:
36102.7/15421.5

2.341062801932367

In [11]:
15421.5 / 14387.85

1.0718418665749225